# Data Wrangling 1.5 Solutions

In [ ]:
import openpyxl

import psycopg2

import json

import csv

import openpyxl

In [ ]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [ ]:
cursor = connection.cursor()

In [ ]:
def my_read_excel_work_book_sheet(excel_file_name, sheet_name, limit):
    "open the excel file, find the sheet_name, print the number of rows specified"
    
    work_book = openpyxl.load_workbook(excel_file_name)
    
    work_sheet = work_book[sheet_name]
                                       
    print("\nExcel Workbook:", excel_file_name)
          
    print("\nWork Sheet:", sheet_name)
    
    print("\nNumber of Columns:", work_sheet.max_column)
          
    print("\nNumber of Rows:", work_sheet.max_row)
    
    work_sheet_list = []
    
    for i in range(1,work_sheet.max_row+1):
        
        row_list = []
        
        for j in range(1,work_sheet.max_column+1):
            
            row_list.append(work_sheet.cell(row = i, column = j).value)
            
        work_sheet_list.append(row_list)
    
    print("\nPrinting the first", limit, "rows :\n")
    
    i = 0
    for row in work_sheet_list:
        print(row)
        i += 1
        if i == limit:
            break

## You try it - read the remaining worksheets in the Excel workbook: Customers, Sales, Line Items, Products, Holidays

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report.xlsx", "Customers", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report.xlsx", "Sales", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report.xlsx", "Line Items", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report.xlsx", "Products", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report.xlsx", "Holidays", 10)

## You try it - extract the remaining worksheets in the workbook into csv files; verify the csv files by reading them:

* Customers to temp_customers_excel.csv
* Sales to temp_sales_excel.csv
* Line Items to temp_line_items.csv
* Products to temp_products_excel.csv
* Holidays to temp_holidays_excel.csv

In [ ]:
def my_export_excel_work_book_sheet_to_csv(excel_file_name, sheet_name, csv_file_name):
    "open the excel file, find the sheet_name, export to csv file "
    
    work_book = openpyxl.load_workbook(excel_file_name)
    
    work_sheet = work_book[sheet_name]
                                       
    print("\nExcel Workbook:", excel_file_name)
          
    print("\nWork Sheet:", sheet_name)
    
    print("\nNumber of Columns:", work_sheet.max_column)
          
    print("\nNumber of Rows:", work_sheet.max_row)
    
    work_sheet_list = []
    
    for i in range(1,work_sheet.max_row+1):
        
        row_list = []
        
        for j in range(1,work_sheet.max_column+1):
            
            row_list.append(work_sheet.cell(row = i, column = j).value)
            
        work_sheet_list.append(row_list)
    
    print("\nExtracting Work Sheet", sheet_name, "to", csv_file_name, "\n")
    
    f = open(csv_file_name, "w")    
    writer = csv.writer(f)
    writer.writerows(work_sheet_list)
    f.close()

In [ ]:
my_export_excel_work_book_sheet_to_csv("temp_sale_report.xlsx", "Customers", "temp_customers_excel.csv")

In [ ]:
my_export_excel_work_book_sheet_to_csv("temp_sale_report.xlsx", "Sales", "temp_sales_excel.csv")

In [ ]:
my_export_excel_work_book_sheet_to_csv("temp_sale_report.xlsx", "Line Items", "temp_line_items_excel.csv")

In [ ]:
my_export_excel_work_book_sheet_to_csv("temp_sale_report.xlsx", "Products", "temp_products_excel.csv")

In [ ]:
my_export_excel_work_book_sheet_to_csv("temp_sale_report.xlsx", "Holidays", "temp_holidays_excel.csv")

In [ ]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [ ]:
my_read_csv_file("temp_customers_excel.csv", 25)

In [ ]:
my_read_csv_file("temp_sales_excel.csv", 25)

In [ ]:
my_read_csv_file("temp_line_items_excel.csv", 25)

In [ ]:
my_read_csv_file("temp_products_excel.csv", 25)

In [ ]:
my_read_csv_file("temp_holidays_excel.csv", 25)

## You try it -  add code cells to the above code to add worksheets for Customers, Sales, Line Items, Products, and Holidays; verify by reading the sheets

In [ ]:
def my_create_excel_sheet_from_query(work_sheet, query):
    "given a work sheet name and query, execute the query and put the results into the sheet"
    
    connection.rollback()
    
    cursor.execute(query)

    connection.rollback()
    
    column_list = [d[0] for d in cursor.description]
    
    row_number = 1
    column_number = 1
    
    for column_name in column_list:
        
        work_sheet.cell(row = row_number, column = column_number, value=column_name)
        
        openpyxl.worksheet.dimensions.ColumnDimension(work_sheet, index=chr(ord('A')+row_number-1), bestFit=True)
        
        column_number += 1
    
    rows = cursor.fetchall()
    
    row_number = 2
    
    for row in rows:
        
        for column_number in range(1, len(column_list)+1):
        
            work_sheet.cell(row = row_number, column = column_number, value=row[column_number-1])
        
        
        row_number += 1


In [ ]:
work_book = openpyxl.Workbook()


In [ ]:
work_sheet = work_book.active
work_sheet.title = "Sales Report"

query = """

select s.store_id,
       s.city as store,
       (lpad(sa.store_id::text, 3, '0') || '-' || lpad(sa.sale_id::text, 9, '0')) as receipt_number,
       sa.sale_date,
       extract(dow from sa.sale_date) as dow,
       to_char(sa.sale_date, 'Day') as day_of_week,
       to_char(sa.sale_date, 'Month') as month,
       l.line_item_id,
       l.product_id,
       p.description as product_name,
       l.quantity,
       l.quantity * 12 as line_item_amount,
       cu.customer_id,
       cu.first_name,
       cu.last_name,
       (cu.first_name || ' ' || cu.last_name) as first_last_name,
       (cu.last_name || ', ' || cu.first_name) as last_first_name,
       cu.street,
       cu.city,
       cu.state,
       cu.zip,
       cu.closest_store_id,
       cu.distance,
       z.latitude,
       z.longitude,
       z.population,
       z.area,
       z.density,
       z.time_zone
from temp_stores as s
     join temp_sales as sa
         on s.store_id = sa.store_id 
     join temp_customers as cu
         on sa.customer_id = cu.customer_id
     join temp_line_items l
         on sa.store_id = l.store_id and sa.sale_id = l.sale_id
     join temp_products p
         on l.product_id = p.product_id
     join zip_codes z
         on cu.zip = z.zip
order by s.store_id, sa.sale_id, l.line_item_id


"""

my_create_excel_sheet_from_query(work_sheet, query)



In [ ]:
work_sheet = work_book.create_sheet("Stores")

query = """

select *
from temp_stores
order by store_id


"""

my_create_excel_sheet_from_query(work_sheet, query)


In [ ]:
work_sheet = work_book.create_sheet("Customers")

query = """

select *
from temp_customers
order by customer_id


"""

my_create_excel_sheet_from_query(work_sheet, query)


In [ ]:
work_sheet = work_book.create_sheet("Sales")

query = """

select *
from temp_sales
order by store_id, sale_id


"""

my_create_excel_sheet_from_query(work_sheet, query)


In [ ]:
work_sheet = work_book.create_sheet("Line Items")

query = """

select *
from temp_line_items
order by store_id, sale_id, line_item_id


"""

my_create_excel_sheet_from_query(work_sheet, query)


In [ ]:
work_sheet = work_book.create_sheet("Products")

query = """

select *
from temp_products
order by product_id


"""

my_create_excel_sheet_from_query(work_sheet, query)

In [ ]:
work_sheet = work_book.create_sheet("Holidays")

query = """

select *
from holidays
order by holiday_date


"""

my_create_excel_sheet_from_query(work_sheet, query)

In [ ]:
work_book.save("temp_sale_report_2.xlsx")

In [ ]:
def my_read_excel_work_book_list_sheets(excel_file_name):
    "open the excel file, list the sheets"
    
    work_book = openpyxl.load_workbook(excel_file_name)
    
    print("\nExcel Workbook", excel_file_name, "list of work sheets:\n")
    
    for s in work_book.sheetnames:
        print(s)
        

In [ ]:
my_read_excel_work_book_list_sheets("temp_sale_report_2.xlsx")

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report_2.xlsx", "Sales Report", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report_2.xlsx", "Stores", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report_2.xlsx", "Customers", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report_2.xlsx", "Sales", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report_2.xlsx", "Line Items", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report_2.xlsx", "Products", 10)

In [ ]:
my_read_excel_work_book_sheet("temp_sale_report_2.xlsx", "Holidays", 10)